In [1]:
import argparse
from typing import Optional
from dotenv import load_dotenv
import os
from tqdm import tqdm
import numpy as np

# torch
import torch

# init hugging face
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain, LLMMathChain
from langchain.utilities import SearchApiAPIWrapper


from unidecode import unidecode

from utils import compile_fp, convert_units, accuracy_metric
import json

load_dotenv()

True

In [2]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def chat(messages, model_name="gpt-4"):
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model_name,
    )

    return chat_completion.choices[0].message.content

chat([{"role": "user", "content": "Say Hello World"}])

'Hello World'

In [ ]:
# ## PROMPTS

# INPUT_PROMPT = PromptTemplate(
#                         input_variables=["context", "question"],
#                         template="""CONTEXT:=\n{context}\nQUESTION:={question}\n"""
#                     )

# SYSTEM_PROMPT_STRING = """
# You are a helpful assistant tasked with answering estimation questions using provided information.
# You are provided with contextual information which contains facts that is relevant to answering the estimation question.
# You are also provided with a question to answer.
# You must write a program that outputs the answer to the question.
# Your program should ask sub-questions to help you answer the question.
# When you answer a sub-question you must provide your reasoning for the answer.
# You can only use the provided contextual information to answer the question and sub-questions.
# Do not use any other information to answer the question or sub-questions.
# If you are unable to answer a question or sub-question, you can set the answer to be the string 'UNK' and proceed with your program as if 'UNK' was the correct answer.
# Some example programs are provided for you below.
# """
# FEW_SHOT_PROMPTS = [
# """Input:
# CONTEXT:=
# F1: The volume of the penny is 0.93 in**3.
# F2: The surface area of a sphere is 4*pi*r**2 where r is the radius of the sphere.
# F3: The volume of a sphere is 4/3*pi*r**3 where r is the radius of the sphere.
# QUESTION:=If I were to take an English penny, melt it down, then somehow stretch it out into a perfect hollow sphere 1 atom thick, how large would this sphere be?

# ```python
# # Q0: If I were to take an English penny, melt it down, then somehow stretch it out into a perfect hollow sphere 1 atom thick, how large would this sphere be?
# # Q1: What is the volume of a single penny?
# A1=0.93 # volume of a single penny (in**3); R: F1 tells us what the volume of a penny is.
# # Q2: What is the surface area of a hollow sphere 1 atom thick which is constructed from a melted penny?
# A2=0.93 # surface area of a hollow 1 atom thick sphere constructed from a melted penny (in**2) R: If a penny is melted down and stretched out into a hollow sphere 1 atom thick, the surface area of the sphere is the same as the volume of the penny.
# # Q3: What is the radius of the sphere given its surface area?
# A3=(A2/(4*np.pi))**(1/2) # radius of the sphere (in) R: F2 implies that the radius of the sphere is the square root of the surface area of the sphere divided by 4*pi.
# A0=(4/3)*np.pi*(A3 ** 3) # volume of the sphere (in**3) R: F3 tells us the volume of the sphere is 4/3*pi*r**3 where r is the radius.
# ```""",
# """Input:
# CONTEXT:=
# F1: The volume of a single air molecule is 3e-24 cubic meter.
# QUESTION:=How many air molecules are in this lecture hall?

# ```python
# # Q0: How many air molecules are in this lecture hall?
# # Q1: What is the volume of the lecture hall?
# A1='UNK' # volume of the lecture hall (m**3) R: Unknown Quantity
# # Q2: What is the volume of a single air molecule?
# A2=3e-24 # volume of single air molecule (m**3) R: F1 tells us the volume of a single air molecule.
# A0=A1/A2 # number of air molecules in lecture hall (molecules) R: The number of air molecules in the lecture hall is the volume of the lecture hall divided by the volume of a single air molecule.
# ```""",
# """Input:
# CONTEXT:=

# QUESTION:=A cure for all cancers would increase the average American lifespan by how many years?

# ```python
# # Q0: A cure for all cancers would increase the average American lifespan by how many years?
# # Q1: What is the average American lifespan?
# A1='UNK' # average American lifespan (years) R: Unknown Quantity
# # Q2: How much does cancer reduce the average lifespan by?
# A2='UNK' # amount cancer reduces average lifespan by (years) R: Unknown Quantity
# A0=A1+A2 # average lifespan with cancer (years) R: The average lifespan with a cure for cancer is the average American lifespan plus the amount that cancer reduces the average lifespan by.
# ```""",
# """Input:
# CONTEXT:=
# F1: The mass of a queen size mattress is 140 pounds.
# F2: The mass of the sun is 4.3e+30 pounds.
# F3: The ratio of mass of Betelgeuse to mass of sun is 18
# QUESTION:=What is the number of queen-size mattresses it would take to fill the star Betelgeuse?

# ```python
# # Q0: What is the number of queen-size mattresses it would take to fill the star Betelgeuse which has 18 times the mass of the Sun?
# # Q1: What is the mass of a queen size mattress?
# A1=140 # mass of queen size mattress (pounds) R: F1 tells us the mass of a queen size mattress.
# # Q2: What is the mass of Betelgeuse?
# # Q3: What is the mass of the sun?
# A3=4.3e+30 # mass of the sun (pounds) R: F2 tells us the mass of the sun.
# # Q4: What is the ratio of the mass of Betelgeuse to the mass of the sun?
# A4=18 # ratio of mass of Betelgeuse to mass of the sun (dimensionless) R: F3 tells us the ratio of the mass of Betelgeuse to the mass of the sun.
# A2=A3*A4 # mass of Betelgeuse (pounds) R: The mass of Betelgeuse is the mass of the sun multiplied by the ratio of the mass of Betelgeuse to the mass of the sun.
# A0=A2/A1 # number of queen size mattresses it would take to fill Betelgeuse (mattresses) R: The number of queen size mattresses it would take to fill Betelgeuse is the mass of Betelgeuse divided by the mass of a queen size mattress.
# ```"""
# ]
# FEW_SHOT_PROMPT_STRING = "\n\n".join(FEW_SHOT_PROMPTS)
# extra_prompts = [
# """Input:
# CONTEXT:=
# F1: It takes around 5 seconds to pluck a single strand of hair.
# F2: The entire human body has 5e+6 hair follicles.
# QUESTION:=How long would it take to pluck each hair out of your body one at a time?

# ```python
# # Q0: How long would it take to pluck each hair out of your body one at a time?
# # Q1: How long does it take to pluck a single strand of hair?
# A1=5 # time to pluck single strand of hair (s) R: F1 tells us how long it takes to pluck a single strand of hair.
# # Q2: How many hairs do we have on our body?
# A2=5e+6 # number of hairs on body (hairs) R: F2 tells us how many hairs we have on our body.
# A0=A1*A2 # time to pluck all hairs on body (s) R: The time to pluck all hairs on our body is the time to pluck a single strand of hair multiplied by the number of hairs on our body.
# ```""",
# ]

# COMBINED_PROMPT = PromptTemplate(
#     template=f"{SYSTEM_PROMPT_STRING}\n\n{FEW_SHOT_PROMPT_STRING}\n\n{INPUT_PROMPT.template}",
#     input_variables=INPUT_PROMPT.input_variables,
# )

In [24]:
## PROMPTS

INPUT_PROMPT = PromptTemplate(
                        input_variables=["question"],
                        template="""QUESTION:={question}\n"""
                    )


SYSTEM_PROMPT_STRING = """
You are a helpful assistant tasked with answering estimation questions.
The user will first ask an estimation question to answer and at the end of the question the user will indicate the units that they want the answer to be in or dimensionless if the answer has no units.

Your initial answer will contain two parts: a summary and a program.
SUMMARY:= First you should write a concise summary of any hypothetical scenarios which the question sets up as well as what is the exact quantity that the question is asking you to estimate.
PROGRAM:=
Next you must write a Python program that answers the estimation question and stores that answer in the variable `A0`.
Your program will have the following structure:
# Q_: Your program should ask sub-questions to help you answer the question.
# R_: After asking each sub-question, you must provide your reasoning for why the sub-question is relevant to our goal of answering the main question.
... You are then encouraged to decompose the sub-question even further unless you are confident that you know the numerical answer to the sub-question ...
# F_: Before answer each sub-question, you will provide a fact that you are referencing to answer the sub-question. The fact must be verifiable by a third party.
A_= You will then answer these sub-questions by assigning the answer to the variable `A_` where the number after the `A` corresponds to the number after the `Q` of the sub-question. # then put the units in a comment after the variable assignment
Please decompose all sub-questions as much as possible to ensure that all quantities are estimated correctly and the math to recompose the quantities is as simple as possible.
Ensure that all quantities use units which are consistent with the units for the final answer.
Additionally, make sure all numerical quantities used in your Python code are written in scientific notation.

After you provide this initial answer, the user will then provide a list of contextual information which they want you to integrate into your answer.
CONTEXT:=
The user will provide this contextual information as a bullet point list of facts.

Your response to the user will be a revision of your initial program.
REVISED PROGRAM:=
This program will only consist of facts (F_) and answers (A_) from your original program.
However, you must revise the initial numerical values used in the program with the new contextual information provided by the user.
You do not need to keep all sub-questions from your original program if they become unnecessary to the estimation question given the new contextual information.

In the following messages, you are provided with verified examplar conversations between the user and the assissant.
Lastly, you will be provided with a user question and you must continue the conversation in the same format as the previous examplar conversations.
"""
FEW_SHOT_PROMPTS = [
"""Input:
QUESTION:=If I were to take an English penny, melt it down, then somehow stretch it out into a perfect hollow sphere 1 atom thick, how large would this sphere be? (units: in**3)

SUMMARY:= We take a penny, melt it down, and stretch it out into a hollow sphere 1 atom thick. We want to compute the volume of this sphere.
PROGRAM:=```python
# Q0: If I were to take an English penny, melt it down, then somehow stretch it out into a perfect hollow sphere 1 atom thick, how large would this sphere be?
# R0: This is the main question we want to answer.
# Q1: What is the volume of a single penny?
# R1: To compute the volume of the hollow sphere, we need the original volume of the penny since it is what gets melted down and turned into the sphere.
# F1: The volume of a penny is 0.93 in**3.
A1=9.3e-1 # (in**3)
# Q2: What is the surface area of a hollow sphere 1 atom thick which is constructed from a melted penny?
# R2: We only know the volume of the penny and we need to translate that information into information about the sphere.
# F2: The surface area of a hollow sphere 1 atom thick constructed from a melted penny is the same as the volume of the penny.
A2=9.3e-1 # (in**2)
# Q3: What is the radius of the sphere given its surface area?
# R3: To compute the volume of the sphere given its surface area, we first need to compute the radius of the sphere.
# F3: The radius of a sphere is the square root of the surface area of the sphere divided by 4*pi.
A3=(A2/(4*np.pi))**(1/2) # (in)
# F0: The volume of a sphere is 4/3*pi*r**3 where r is the radius of the sphere.
A0=(4/3)*np.pi*(A3 ** 3) # (in**3)
```
""",
# """Input:
# QUESTION:=How many air molecules are in this lecture hall?

# SUMMARY:= We have a lecture hall and we want to compute the number of air molecules inside of it.
# PROGRAM:=```python
# # Q0: How many air molecules are in this lecture hall?
# # R0: This is the main question we want to answer.
# # Q1: What is the volume of an average lecture hall?
# # R1: To get a sense of how many air molecules we can fit in this lecture hall, we first need to know its volume which can be approximated by the volume of an average lecture hall.
# # F1: The volume of an average lecture hall is 1000 cubic meters.
# A1=1000 # (m**3)
# # Q2: What is the volume of a single air molecule?
# # R2: To compute the number of air molecules in some space with a known volume, we need to know the volume of a single air molecule.
# # F2: The volume of a single air molecule is 3e-24 cubic meter.
# A2=3e-24 # (m**3)
# # F0: The number of air molecules in the lecture hall is the volume of the lecture hall divided by the volume of a single air molecule.
# A0=A1/A2 # (molecules)
# ```
# """,
"""Input:
QUESTION:=How much would the sea level rise if all the ice melted? (units: mi)

SUMMARY:= All of the ice on Earth has melted and flown into the ocean. We want to compute how much the sea level will rise.
PROGRAM:=
# Q0: How much would the sea level rise if all the ice melted?
# R0: This is the main question we want to answer.
# Q1: What is the volume of water that will be added to the ocean?
# R1: If we know the volume of water that will be added to the ocean, we can compute the sea level rise by dividing the volume of water by the surface area of the ocean.
# Q2: What is the volume of ice on Earth that is on land?
# R2: The volume of water that will be added to the ocean is the same as the volume of ice on Earth that is on land since that ice will melt and flow into the ocean.
# Q3: What is the volume of ice in Antarctica?
# R3: The majority of the ice on land is in Antarctica so it is a good approximation to use the volume of ice in Antarctica as the volume of ice on Earth that is on land.
# Q4: What is the thickness of the ice in Antarctica?
# R4: To compute the volume of ice in Antarctica, we need to know the thickness of the ice because we can multiply that by its surface area to get its volume.
# F4: The thickness of the ice in Antarctica is 4.8 km. We must convert this into mi by multiplying by 0.621371.
A4=0.621371 * 4.8e+0 # (mi)
# Q5: What is the surface area of Antarctica?
# R5: To compute the volume of ice in Antarctica, we need to know the surface area of Antarctica because we can multiply that by its thickness to get its volume.
# Q6: What fraction of the Earth's surface is Antarctica?
# R6: To compute the surface area of Antarctica, we need to know what fraction of the Earth's surface is Antarctica because we can multiply that by the surface area of the Earth to get the surface area of Antarctica.
# F6: Antarctica is 1/30th of the Earth's surface.
A6=1/30 # (dimensionless)
Q7: What is the surface area of the Earth?
# R7: To compute the surface area of Antarctica, we need to know the surface area of the Earth because we can multiply that by the fraction of the Earth's surface that is Antarctica to get the surface area of Antarctica.
# F7: The surface area of the Earth is 1.97e+8 mi**2.
A7=1.97e+8 # (mi**2)
# F5: The surface area of Antarctica is the fraction of the Earth's surface that is Antarctica multiplied by the surface area of the Earth.
A5=A6*A7 # (mi**2)
# F3: The volume of ice in Antarctica is the surface area of Antarctica multiplied by the thickness of the ice in Antarctica.
A3=A5*A4 # (mi**3)
# F2: The volume of ice on Earth is approximated by the volume of ice in Antarctica.
A2=A3 # (mi**3)
# F1: The volume of water that will be added to the ocean is the same as the volume of ice on Earth.
A1=A2 # (mi**3)
# Q8: What is the surface area of the ocean?
# R8: To compute the sea level rise, we need to know the surface area of the ocean because we can divide the volume of water that will be added to the ocean by the surface area of the ocean to get the sea level rise.
# F8: The surface area of the ocean is 361 million km**2. We must convert this into mi**2 by multiplying by 0.386102.
A8=0.386102 * 361e+8 # (mi**2)
# F0: The sea level rise is the volume of water that will be added to the ocean divided by the surface area of the ocean.
A0=A1/A8 # (mi)
"""
]
FEW_SHOT_PROMPT_STRING = "\n\n".join(FEW_SHOT_PROMPTS)
extra_prompts = [
"""Input:
QUESTION:=How long would it take to pluck each hair out of your body one at a time?

CONTEXT:=
F1: It takes around 5 seconds to pluck a single strand of hair.
F2: The entire human body has 5e+6 hair follicles.
PROGRAM:=
# Q0: How long would it take to pluck each hair out of your body one at a time?
# Q1: How long does it take to pluck a single strand of hair?
A1=5 # time to pluck single strand of hair (s) R: F1 tells us how long it takes to pluck a single strand of hair.
# Q2: How many hairs do we have on our body?
A2=5e+6 # number of hairs on body (hairs) R: F2 tells us how many hairs we have on our body.
A0=A1*A2 # time to pluck all hairs on body (s) R: The time to pluck all hairs on our body is the time to pluck a single strand of hair multiplied by the number of hairs on our body.
""",
"""Input:
QUESTION:=A cure for all cancers would increase the average American lifespan by how many years?

PROGRAM:=
# Q0: A cure for all cancers would increase the average American lifespan by how many years?
# Q1: What is the average American lifespan?
# R1: The average American lifespan is 78 years.
A1=78 # (years)
# Q2: How much does cancer reduce the average lifespan by?
# R2: Cancer reduces the average lifespan by 5 years.
A2=5 # (years)
# R0: To compute the average lifespan with a cure for cancer, take the average American lifespan plus the amount that cancer reduces the average lifespan by.
A0=A1+A2 # (years)
""",
"""Input:
QUESTION:=What is the number of queen-size mattresses it would take to fill the star Betelgeuse?

PROGRAM:=
# Q0: What is the number of queen-size mattresses it would take to fill the star Betelgeuse which has 18 times the mass of the Sun?
# Q1: What is the mass of a queen size mattress?
# R1: The mass of a queen size mattress is 140 pounds.
A1=140 # (pounds)
# Q2: What is the mass of Betelgeuse?
# Q3: What is the mass of the sun?
# R3: The mass of the sun is 4.3e+30 pounds.
A3=4.3e+30 # (pounds)
# Q4: What is the ratio of the mass of Betelgeuse to the mass of the sun?
# The ratio of mass of Betelgeuse to mass of sun is 18
A4=18 # (dimensionless)
# R2: The mass of Betelgeuse is the mass of the sun multiplied by the ratio of the mass of Betelgeuse to the mass of the sun.
A2=A3*A4 # (pounds)
# R0: The number of queen size mattresses it would take to fill Betelgeuse is the mass of Betelgeuse divided by the mass of a queen size mattress.
A0=A2/A1 # (mattresses)
""",
]

COMBINED_PROMPT = PromptTemplate(
    template=f"{SYSTEM_PROMPT_STRING}\n\n{FEW_SHOT_PROMPT_STRING}\n\n{INPUT_PROMPT.template}",
    input_variables=INPUT_PROMPT.input_variables,
)

In [21]:
class SamplePredictor:
    def __init__(self, temperature=0, prompt=COMBINED_PROMPT):

        # self.prompt = prompt

        # llm = OpenAI(
        #         model="gpt-3.5-turbo",
        #         temperature=temperature,
        #         openai_api_key=os.getenv("OPENAI_API_KEY")
        #     )
        # self.llm_chain = LLMChain(prompt=self.prompt, llm=llm)

        # self.chat_model = ChatOpenAI(
        #         model="gpt-4",
        #         temperature=temperature,
        #         openai_api_key=os.getenv("OPENAI_API_KEY")
        #     )
        # self.llm_chain = LLMChain(llm=chat_llm)
        self.chat = chat


    @staticmethod
    def eval(question, preds, answer, num_tokens=None, verbose=True):
        try:
            # program = program[program.find("```python")+len("```python"):program.rfind("```")]
            summary = preds.split("SUMMARY:=")[-1].split("PROGRAM:=")[0].strip('\n')
            program = preds.split("PROGRAM:=")[-1].strip('\n')
            program = program[program.find("```python")+len("```python"):program.rfind("```")]
            print(program)
            loc = {}
            # exec(program.replace("'UNK'", "100"), globals(), loc)
            exec(program, globals(), loc)
            print(loc)
            compiled_out = loc["A0"]
            # compiled_units = program.split(")")[-2].split("(")[-1]

            prediction = {
                    "question": question,
                    "correct_answer": answer,
                    "summary": summary,
                    "program": program,
                    "raw_outputs": preds,
                    "compiled_answer": compiled_out,
                    # "compiled_units": compiled_units
                }

            compiled_acc = accuracy_metric(answer, compiled_out)
            parsable = 1
        except:
            prediction = {
                    "question": question,
                    "correct_answer": answer,
                    "raw_outputs": preds
                }
            compiled_acc = 0
            parsable = 0
        if verbose:
            print("Question: {}; \nCorrect Answer: {}".format(prediction["question"], prediction["correct_answer"]))
            if num_tokens is not None:
                print("Number of Tokens in Response: ", num_tokens)

            if len(prediction) > 3:
                print("Compiled Answer is: {} \nSummarized Problem is: {}\nProgram:\n{}".format(prediction['compiled_answer'], prediction['summary'], prediction['program']))
            else:
                print("Unable to parse output; Outputting Raw Output:\n{}".format(prediction["raw_outputs"]))
        return compiled_acc, parsable, prediction

    def ask(self, question, answer, context="", verbose=True):
        # preds = self.llm_chain.run(context=context, question=question)
        # preds = self.llm_chain.run(question=question)
        messages = [{"role": "system", "content": SYSTEM_PROMPT_STRING}]
        for few_shot_prompt in FEW_SHOT_PROMPTS:
            user_message = few_shot_prompt.split("QUESTION:=")[-1].split("\n")[0]
            messages.append({"role": "user", "content": user_message})
            assistant_message = "\n" + few_shot_prompt.split("\n\n")[1].strip('\n')
            messages.append({"role": "assistant", "content": assistant_message})
        messages.append({"role": "user", "content": question})

        preds = self.chat(messages)
        # num_tokens = self.llm_chain._get_num_tokens(preds)
        return self.eval(question, preds, answer, num_tokens=None, verbose=verbose)

    def predict(self, dataset, N, include_context=True, verbose=False):
        compiled_acc = 0
        parsable = 0
        shuffled_idxs = np.arange(len(dataset))
        np.random.shuffle(shuffled_idxs)
        print("Shuffled Indices w/ fixed seed: ", shuffled_idxs[:10])
        predictions = []
        for i in tqdm(range(N)):
            entry = dataset[shuffled_idxs[i]]
            question = entry["question"]
            answer = entry["answer"]
            context = entry["context"] if include_context else ""
            compiled_acc_i, parsable_i, prediction = self.ask(question, answer, context, verbose=verbose)
            compiled_acc += compiled_acc_i
            parsable += parsable_i
            predictions.append(prediction)

        return compiled_acc/N, parsable/N, predictions


In [22]:
predictor = SamplePredictor(temperature=0.0)

In [10]:
question = """If the known universe were placed on a microscope slide, what magnification would be necessary to see Earth? (units: dimensionless)"""
answer = '3.10E+59'
context = """"""

predictor.ask(question, answer, context)


Question: If the known universe were placed on a microscope slide, what magnification would be necessary to see Earth? (units: dimensionless); 
Correct Answer: 3.10E+59
Unable to parse output; Outputting Raw Output:
SUMMARY:= We are shrinking the known universe down to fit on a microscope slide, and wanted to know what magnification is needed to clearly see the Earth. 

PROGRAM:=```python
# Q0: If the known universe were placed on a microscope slide, what magnification would be necessary to see Earth?
# R0: This is the main question we want to answer.
# Q1: What is the diameter of the known universe?
# R1: To compute the magnification necessary, we need to know the scale at which we've shrunk the universe to fit onto a microscope slide.
# F1: The diameter of the observable universe is about 93 billion light-years, which in meters would be approximately 8.8x10^26 meters. 
A1 = 8.8e26 # meters
# Q2: What is the diameter of the microscope slide?
# R2: We need to know the size of the micro

(0,
 0,
 {'question': 'If the known universe were placed on a microscope slide, what magnification would be necessary to see Earth? (units: dimensionless)',
  'correct_answer': '3.10E+59',
  'raw_outputs': "SUMMARY:= We are shrinking the known universe down to fit on a microscope slide, and wanted to know what magnification is needed to clearly see the Earth. \n\nPROGRAM:=```python\n# Q0: If the known universe were placed on a microscope slide, what magnification would be necessary to see Earth?\n# R0: This is the main question we want to answer.\n# Q1: What is the diameter of the known universe?\n# R1: To compute the magnification necessary, we need to know the scale at which we've shrunk the universe to fit onto a microscope slide.\n# F1: The diameter of the observable universe is about 93 billion light-years, which in meters would be approximately 8.8x10^26 meters. \nA1 = 8.8e26 # meters\n# Q2: What is the diameter of the microscope slide?\n# R2: We need to know the size of the micr

In [23]:
preds = """SUMMARY:= We are shrinking the known universe down to fit on a microscope slide, and wanted to know what magnification is needed to clearly see the Earth.

PROGRAM:=```python
# Q0: If the known universe were placed on a microscope slide, what magnification would be necessary to see Earth?
# R0: This is the main question we want to answer.
# Q1: What is the diameter of the known universe?
# R1: To compute the magnification necessary, we need to know the scale at which we've shrunk the universe to fit onto a microscope slide.
# F1: The diameter of the observable universe is about 93 billion light-years, which in meters would be approximately 8.8x10^26 meters.
A1 = 8.8e26 # meters
# Q2: What is the diameter of the microscope slide?
# R2: We need to know the size of the microscope slide to understand what scale the universe has been shrunk to.
# F2: A standard microscope slide has a width of 25 millimeters, or 0.025 meters.
A2 = 2.5e-2 # meters
# Q3: What is the diameter of Earth?
# R3: To be able to calculate the magnification required to see Earth, we need to know Earth's diameter.
# F3: The mean diameter of Earth is approximately 12,742 kilometers, which in meters would be approximately 1.2742x10^7 meters.
A3 = 1.2742e7 # meters
# Q4: What is the diameter of Earth when the universe is shrunken to the size of a microscope slide?
# R4: Knowing the diameter of Earth at scale will help us calculate the magnification needed.
# F4: The scale factor is equal to the diameter of the slide divided by the diameter of the universe.  The diameter of Earth at scale is equal to the diameter of Earth multiplied by the scale factor.
scale_factor = A2 / A1
A4 = A3 * scale_factor # meters
# Q5: What is the smallest distance the human eye can resolve?
# R5: To calculate the magnification required, we have to know this distance because it is the lower limit for what we have to magnify to.
# F5: The smallest distance the human eye can resolve is approximately 0.2 millimeters or 0.0002 meters.
A5 = 2e-4 # meters
# F0: The required magnification is the smallest distance the human can resolve divided by the diameter of Earth when the universe is shrunken to the size of a microscope slide.
A0 = A5 / A4 # dimensionless
A0
```"""
predictor.eval(question, preds, answer, verbose=True)


# Q0: If the known universe were placed on a microscope slide, what magnification would be necessary to see Earth?
# R0: This is the main question we want to answer.
# Q1: What is the diameter of the known universe?
# R1: To compute the magnification necessary, we need to know the scale at which we've shrunk the universe to fit onto a microscope slide.
# F1: The diameter of the observable universe is about 93 billion light-years, which in meters would be approximately 8.8x10^26 meters.
A1 = 8.8e26 # meters
# Q2: What is the diameter of the microscope slide?
# R2: We need to know the size of the microscope slide to understand what scale the universe has been shrunk to.
# F2: A standard microscope slide has a width of 25 millimeters, or 0.025 meters.
A2 = 2.5e-2 # meters
# Q3: What is the diameter of Earth?
# R3: To be able to calculate the magnification required to see Earth, we need to know Earth's diameter.
# F3: The mean diameter of Earth is approximately 12,742 kilometers, which in

(0.0,
 1,
 {'question': 'If the known universe were placed on a microscope slide, what magnification would be necessary to see Earth? (units: dimensionless)',
  'correct_answer': '3.10E+59',
  'summary': ' We are shrinking the known universe down to fit on a microscope slide, and wanted to know what magnification is needed to clearly see the Earth.',
  'program': "\n# Q0: If the known universe were placed on a microscope slide, what magnification would be necessary to see Earth?\n# R0: This is the main question we want to answer.\n# Q1: What is the diameter of the known universe?\n# R1: To compute the magnification necessary, we need to know the scale at which we've shrunk the universe to fit onto a microscope slide.\n# F1: The diameter of the observable universe is about 93 billion light-years, which in meters would be approximately 8.8x10^26 meters.\nA1 = 8.8e26 # meters\n# Q2: What is the diameter of the microscope slide?\n# R2: We need to know the size of the microscope slide to un

In [13]:
program = """
# Q0: If the known universe were placed on a microscope slide, what magnification would be necessary to see Earth?
# R0: This is the main question we want to answer.
# Q1: What is the diameter of the known universe?
# R1: To compute the magnification necessary, we need to know the scale at which we've shrunk the universe to fit onto a microscope slide.
# F1: The diameter of the observable universe is about 93 billion light-years, which in meters would be approximately 8.8x10^26 meters.
A1 = 8.8e26 # meters
# Q2: What is the diameter of the microscope slide?
# R2: We need to know the size of the microscope slide to understand what scale the universe has been shrunk to.
# F2: A standard microscope slide has a width of 25 millimeters, or 0.025 meters.
A2 = 2.5e-2 # meters
# Q3: What is the diameter of Earth?
# R3: To be able to calculate the magnification required to see Earth, we need to know Earth's diameter.
# F3: The mean diameter of Earth is approximately 12,742 kilometers, which in meters would be approximately 1.2742x10^7 meters.
A3 = 1.2742e7 # meters
# Q4: What is the diameter of Earth when the universe is shrunken to the size of a microscope slide?
# R4: Knowing the diameter of Earth at scale will help us calculate the magnification needed.
# F4: The scale factor is equal to the diameter of the slide divided by the diameter of the universe.  The diameter of Earth at scale is equal to the diameter of Earth multiplied by the scale factor.
scale_factor = A2 / A1
A4 = A3 * scale_factor # meters
# Q5: What is the smallest distance the human eye can resolve?
# R5: To calculate the magnification required, we have to know this distance because it is the lower limit for what we have to magnify to.
# F5: The smallest distance the human eye can resolve is approximately 0.2 millimeters or 0.0002 meters.
A5 = 2e-4 # meters
# F0: The required magnification is the smallest distance the human can resolve divided by the diameter of Earth when the universe is shrunken to the size of a microscope slide.
A0 = A5 / A4 # dimensionless
A0
"""
exec(program)

In [12]:
A0

5.5250353162768794e+17

In [ ]:
question = "If all but 1 million people on Earth died, how far in kilometers (on average) would you have to walk to meet someone?\n\nSUMMARY:= We want to compute the average distance in kilometers that one would have to walk to meet someone if all but 1 million people on Earth died.\nPROGRAM:=\n# Q0: If all but 1 million people on Earth died, how far in kilometers (on average) would you have to walk to meet someone?"

preds = """
SUMMARY:= We want to compute the average distance in kilometers that one would have to walk to meet someone if all but 1 million people on Earth died.
PROGRAM:=
# Q0: If all but 1 million people on Earth died, how far in kilometers (on average) would you have to walk to meet someone?
# R0: This is the main question we want to answer.
# Q1: What is the total land area of Earth?
# R1: To compute the average distance one would have to walk to meet someone, we need to know the total land area of Earth.
# F1: The total land area of Earth is 148,939,063 km**2.
A1=148939063 # (km**2)
# Q2: How many people are left on Earth if all but 1 million people died?
# R2: To compute the average distance one would have to walk to meet someone, we need to know how many people are left on Earth.
# F2: If all but 1 million people on Earth died, then there would be 1 million people left on Earth.
A2=1000000 # (people)
# F0: The average distance one would have to walk to meet someone is the total land area of Earth divided by the number of people left on Earth.
A0=A1/A2 # (km)
"""

predictor.eval(question, preds, answer)

In [ ]:
llm = OpenAI(temperature=0)
llm("What is the energy needed to evaporate the oceans in calories?")

In [ ]:

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
search = SearchApiAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="Useful for when you need to search for information",
    ),
    # Tool(
    #     name="Calculator",
    #     func=llm_math_chain.run,
    #     description="Useful for when you need to do mathematical calculations",
    # ),
]

self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_TOOLS, verbose=True
)



In [ ]:
self_ask_with_search.run("How much rice can fit in an average sized house?")


In [ ]:
search.run("the average volume of blood donated by a person")

In [ ]:
search.run("1 g/ml = ? kg/m^3")

In [ ]:
llm("What is the formula for the volume of a rectangular prism?")

In [ ]:
llm = OpenAI(temperature=0.5)

llm("""Find a simple formula to answer the following question: How much would the sea level rise if all of the ice on Earth melted?
    You are not allowed to use any numbers in your formula. Use as few variables as possible in your formula.
    After you have found a formula, provide an in-depth explanation of what each variable in your formula represents.
    """)

In [ ]:
variables = "\n\n1. The amount of carbon dioxide exhaled per breath\n2. The amount of oxygen inhaled per breath\n3. The amount of fat in kilograms that needs to be lost\n4. The amount of carbon dioxide produced when fat is burned"
llm(f"You are given the following variables:{variables}\nUsing these variables, devise a mathematical formula to answer the following: How many kilograms of co2 would i have to breath out to lose 1 kg of fat? Refer to the variables by their index in the list above.")

In [ ]:
llm_math_chain("(7.2e+2 m**3) * (1.452e+3 kg/m**3) # (Volume of an average house) * (the density of rice) ")